In [1]:
!python3 --version

Python 3.11.2


In [16]:
import pandas as pd
import numpy as np

In [2]:
# Log into Hugging Face
HUGGINGFACE_TOKEN = ""
!huggingface-cli login --token $HUGGINGFACE_TOKEN

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /Users/verosha/.cache/huggingface/token
Login successful


In [3]:
from huggingface_hub import hf_hub_download
import pandas as pd

/Users/verosha/.pyenv/versions/3.11.2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
REPO_ID = "MelioAI/santander-product-recommendation"
HF_TRAIN_DATASET_NAME = "train_ver2.csv"
HF_TEST_DATASET_NAME = "test_ver2.csv"

In [21]:
## Helper function

# Define a function to map product names to numbers
def map_product_to_number(product):
    product_map = {
        'savings_acc': 0,
        'guarantees': 1,
        'current_acc': 2,
        'derivada_acc': 3,
        'payroll_acc': 4,
        'jnr_acc': 5,
        'más_particular_acc': 6,
        'particular_account': 7,
        'particular_plus_account': 8,
        'short_term_deposits': 9,
        'medium_term_deposits': 10,
        'long_term_deposits': 11,
        'e_acc': 12,
        'funds': 13,
        'mortgage': 14,
        'pensions_plan': 15,
        'loans': 16,
        'taxes': 17,
        'credit_card': 18,
        'securities': 19,
        'home_acc': 20,
        'payroll': 21,
        'pensions': 22,
        'direct_debit': 23
    }
    return product_map.get(product, -1)  # Return -1 if product not found

In [5]:
# Using the Hugging Face Client Library

ds_train = pd.read_csv(
    hf_hub_download(repo_id=REPO_ID, filename=HF_TRAIN_DATASET_NAME, repo_type="dataset")
)

/var/folders/py/00rrgyx10993blywbkynytgc0000gn/T/ipykernel_22297/1856397251.py:3: DtypeWarning: Columns (5,8,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  ds_train = pd.read_csv(


In [6]:
ds_test = pd.read_csv(
    hf_hub_download(repo_id=REPO_ID, filename=HF_TEST_DATASET_NAME, repo_type="dataset")
)

/var/folders/py/00rrgyx10993blywbkynytgc0000gn/T/ipykernel_22297/2311066649.py:1: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  ds_test = pd.read_csv(


In [7]:
ds_train.sample(5)

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,...,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
10832478,2016-02-28,891707,N,ES,V,38,2010-09-15,0.0,65,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
6946228,2015-10-28,238263,N,ES,H,46,2001-04-09,0.0,174,1.0,...,0,0,0,0,0,1,0,1.0,1.0,1
4726537,2015-08-28,1019807,N,ES,V,37,2012-06-15,0.0,38,1.0,...,0,0,0,0,0,1,0,0.0,0.0,0
3813805,2015-07-28,723790,N,ES,H,44,2007-10-06,0.0,93,1.0,...,0,0,0,0,0,0,0,1.0,1.0,1
12569465,2016-04-28,1474622,N,ES,V,26,2015-10-09,0.0,6,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0


### Column Details:
fecha_dato - The table is partitioned for this column

ncodpers - Customer code

ind_empleado - Employee index: A active, B ex employed, F filial, N not employee, P pasive

pais_residencia - Customer's Country residence

sexo - Customer's sex

age - Age

fecha_alta - The date in which the customer became as the first holder of a contract in the bank

ind_nuevo - New customer Index. 1 if the customer registered in the last 6 months.

antiguedad - Customer seniority (in months)

indrel - 1 (First/Primary), 99 (Primary customer during the month but not at the end of the month)

ult_fec_cli_1t - Last date as primary customer (if he isn't at the end of the month)

indrel_1mes	- Customer type at the beginning of the month ,1 (First/Primary customer), 2 (co-owner ),P (Potential),3 (former primary), 4(former co-owner)

tiprel_1mes	- Customer relation type at the beginning of the month, A (active), I (inactive), P (former customer),R (Potential)

indresi	- Residence index (S (Yes) or N (No) if the residence country is the same than the bank country)

indext	- Foreigner index (S (Yes) or N (No) if the customer's birth country is different than the bank country)

conyuemp - Spouse index. 1 if the customer is spouse of an employee

canal_entrada - channel used by the customer to join

indfall	- Deceased index. N/S

tipodom	- Addres type. 1, primary address

cod_prov - Province code (customer's address)

nomprov	- Province name

ind_actividad_cliente - Activity index (1, active customer; 0, inactive customer)

renta - Gross income of the household

segmento - segmentation: 01 - VIP, 02 - Individuals 03 - college graduated

ind_ahor_fin_ult1 - Saving Account

ind_aval_fin_ult1 - Guarantees

ind_cco_fin_ult1 - Current Accounts

ind_cder_fin_ult1 - Derivada Account

ind_cno_fin_ult1 - Payroll Account

ind_ctju_fin_ult1 - Junior Account

ind_ctma_fin_ult1 - Más particular Account

ind_ctop_fin_ult1 - particular Account

ind_ctpp_fin_ult1 - particular Plus Account

ind_deco_fin_ult1 - Short-term deposits

ind_deme_fin_ult1 - Medium-term deposits

ind_dela_fin_ult1 - Long-term deposits

ind_ecue_fin_ult1 - e-account

ind_fond_fin_ult1 - Funds

ind_hip_fin_ult1 - Mortgage

ind_plan_fin_ult1 - Pensions

ind_pres_fin_ult1 - Loans

ind_reca_fin_ult1 - Taxes

ind_tjcr_fin_ult1 - Credit Card

ind_valo_fin_ult1 - Securities

ind_viv_fin_ult1 - Home Account

ind_nomina_ult1 - Payroll

ind_nom_pens_ult1 - Pensions

ind_recibo_ult1	- Direct Debit

Common Columns:
Dataset 1: AB data
Dataset 2: Santander Product Recommendation Ddataset

From Dataset 1 and Dataset 2, some common columns that seem relevant for creating a curated dataset could include:

a) Client Identifier:
Clientnumber (Dataset 1)
ncodpers (Dataset 2)

b)Date and Time Information:
DateID (Dataset 1)
fecha_dato (Dataset 2)/ Change to `cust_start_date_first_holder_contract`

c)Customer Demographics:
Gender (Dataset 1)
sexo (Dataset 2)

Age (common in both)
Age (Dataset 2)
Fecha_alta

Get any dataset in the required format. Use the SPR as an example

1) Look at what the curated dataset looks like: 
2) We need to do a mapping of products

# Common Fields

- a) Client Identifier
- b) Date and Time Information
- c) Customer Demographics
- d) products owned (mapping)

# Step 1: Get the common data (a-c)
- For the SPR dataset

In [9]:
# Convert column names

col_names = {"ncodpers":"cust_id", "ind_empleado":"emp_index","pais_residencia":"cust_country_res",
            "sexo":"sex","fecha_alta":"cust_start_date_first_holder_contract","ind_nuevo":"new_cust_index","antiguedad":"cust_seniority",
            "indrel":"cust_primary_type","ult_fec_cli_1t":"cust_last_primary_date","indrel_1mes":"cust_type_at_start_month",
            "tiprel_1mes":"cust_rel_type_at_start_month","indresi":"residence_index","indext":"foreigner_index",
            "conyuemp":"spouse_index","canal_entrada":"channel_joined", "indfall":"deceased_index", "tipodom":"address_type",
            "cod_prov":"province","nomprov":"province_name", "ind_actividad_cliente":"activity_index","renta":"gross_income",
            "segmento":"cust_category", "ind_ahor_fin_ult1":"savings_acc", "ind_aval_fin_ult1":"guarantees",
            "ind_cco_fin_ult1":"current_acc", "ind_cder_fin_ult1":"derivada_acc", "ind_cno_fin_ult1":"payroll_acc",
            "ind_ctju_fin_ult1":"jnr_acc", "ind_ctma_fin_ult1":"más_particular_acc", "ind_ctop_fin_ult1":"particular_account",
            "ind_ctpp_fin_ult1":"particular_plus_account", "ind_deco_fin_ult1":"short_term_deposits",
            "ind_deme_fin_ult1":"medium_term_deposits", "ind_dela_fin_ult1":"long_term_deposits", "ind_ecue_fin_ult1":"e_acc",
            "ind_fond_fin_ult1":"funds","ind_hip_fin_ult1":"mortgage", "ind_plan_fin_ult1":"pensions_plan", "ind_pres_fin_ult1":"loans",
            "ind_reca_fin_ult1":"taxes", "ind_tjcr_fin_ult1":"credit_card", "ind_valo_fin_ult1":"securities", "ind_viv_fin_ult1":"home_acc",
            "ind_nomina_ult1":"payroll", "ind_nom_pens_ult1": "pensions", "ind_recibo_ult1":"direct_debit"}

ds_train.rename(col_names, axis = 1, inplace = True)
ds_test.rename(col_names, axis = 1, inplace = True)
ds_train.sample(3)

,fecha_dato,cust_id,emp_index,cust_country_res,sex,age,cust_start_date_first_holder_contract,new_cust_index,cust_seniority,cust_primary_type,...,mortgage,pensions_plan,loans,taxes,credit_card,securities,home_acc,payroll,pensions,direct_debit
6820372,2015-10-28,437542,N,ES,H,41,2003-09-30,0.0,145,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
2674081,2015-05-28,1312297,N,ES,H,21,2014-09-22,0.0,10,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
1842996,2015-03-28,1044225,N,ES,H,23,2012-08-06,0.0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0


In [13]:
# pull out the common data: (a-c)

# Define the common columns from Dataset 2 with fecha_alta instead of cust_start_date_first_holder_contract
common_columns = [
    "cust_id", "cust_start_date_first_holder_contract", "sex", "age"
]

# Select only the common columns from ds_train
ds_train_common = ds_train[common_columns]

# Rename columns in ds_train_common
ds_train_common.rename(columns={
    "cust_start_date_first_holder_contract": "date_id"
}, inplace=True)

# Display a sample of the curated dataset
ds_train_common.sample(3)

/var/folders/py/00rrgyx10993blywbkynytgc0000gn/T/ipykernel_22297/2481746361.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds_train_common.rename(columns={


,cust_id,date_id,sex,age
6328635,1122175,2013-02-19,V,70
10124404,1529292,2016-02-02,H,29
12776855,524027,2005-03-19,H,59


In [15]:
# Update gender columns
ds_train_common['sex'].replace({'H': 'M', 'V': 'F'}, inplace=True)

ds_train_common.sample(2)

/var/folders/py/00rrgyx10993blywbkynytgc0000gn/T/ipykernel_22297/3210757816.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds_train_common['sex'].replace({'H': 'M', 'V': 'F'}, inplace=True)


,cust_id,date_id,sex,age
4406364,1017060,2012-05-22,F,26
6001752,802098,2008-10-07,F,42


In [23]:
# Create a list of product names from your dictionary
product_names = list(col_names.values())[22:]  # Skip non-product columns

# Initialize the owned_items column with -1
ds_train['owned_items'] = -1

# Iterate through each product and update owned_items where the customer owns them
for product in product_names:
    ds_train.loc[ds_train[product] == 1, 'owned_items'] = ds_train.loc[ds_train[product] == 1, product].apply(lambda x: map_product_to_number(product))


In [24]:
# Display the DataFrame to verify the results
ds_train[['cust_id', 'owned_items']].head(50)

,cust_id,owned_items
0,1375586,2
1,1050611,2
2,1050612,2
3,1050613,9
4,1050614,2
5,1050615,2
6,1050616,2
7,1050617,2
8,1050619,2
9,1050620,2


In [46]:
# Specify the cust_id you want to check
cust_id_to_check = 1050676  # Replace with the cust_id you want to check

# Filter the dataset for the specified cust_id
customer_ownership = ds_train[ds_train['cust_id'] == cust_id_to_check]

# Display the cust_id and owned_items for the specified customer
print(customer_ownership[['cust_id', 'owned_items']])

          cust_id  owned_items
41        1050676           12
1243198   1050676           12
1847947   1050676           12
2507325   1050676           12
2519083   1050676            2
3176813   1050676           12
4371932   1050676           12
5136532   1050676           12
6053982   1050676           12
6492139   1050676           12
8085975   1050676           12
8187374   1050676           12
9891834   1050676           12
10604207  1050676           23
11786278  1050676           12
12445337  1050676           23
13431218  1050676           12


In [49]:
# Specify the cust_id you want to check
cust_id_to_check = 1050680  # Replace with the cust_id you want to check

# Filter the dataset for the specified cust_id
customer_ownership = ds_train[ds_train['cust_id'] == cust_id_to_check]

# Aggregate unique owned_items into a list
owned_items_list = customer_ownership['owned_items'].unique().tolist()

# Create a DataFrame to display the result
result_df = pd.DataFrame({'cust_id': [cust_id_to_check], 'owned_items': [owned_items_list]})

# Display the result
print(result_df)


   cust_id owned_items
0  1050680        [-1]
